In [2]:
import requests
import math
import pandas as pd
import json
import os

In [3]:
os.system('mkdir -p cache')

1

In [15]:
import re
from tokenizers import normalizers
from tokenizers.normalizers import BertNormalizer, Sequence, Strip
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Whitespace

normalizer = Sequence([BertNormalizer(clean_text=True,
        handle_chinese_chars=True,
        strip_accents=True,
        lowercase=True), Strip()])
pre_tokenizer = pre_tokenizers.Sequence([Whitespace()])

def normalize(x, min_length = 1):
    if not isinstance(x, str):
        return ''
    normalized = normalizer.normalize_str(x)
    normalized = normalized.replace('\n', ' ')
    normalized = re.sub(' +', ' ', normalized)
    return " ".join([e[0] for e in pre_tokenizer.pre_tokenize_str(normalized) if len(e[0]) > min_length])

### Nettoyer lib entreprises pour query

In [5]:
import urllib.parse

In [6]:
companies = pd.read_csv('liste_entreprises.csv',sep=';', index_col=0)
companies.columns

Index(['name', 'source', 'type'], dtype='object')

In [21]:
list_companies = [] 
for company in companies['name']:
    company = company.replace(',','')
    a = urllib.parse.quote(company)
    list_companies.append(a)

In [22]:
list_companies

['GENINDEXE',
 'AMBIENTIC',
 'HYDROCEAN',
 'COVALIA%20INTERACTIVE',
 'Aquitaine%20Science%20Transfert',
 'EXOTRAIL',
 'AKENEO',
 'CENTRE%20DE%20RECHERCHE%20POUR%20L%27ARCHITECTURE%20ET%20LES%20INDUSTRIES%20NAUTIQUES',
 'SATT%20Linksium%20Grenoble%20Alpes',
 'TEAMCAST%20TECHNOLOGY',
 'Atelier%20de%20r%C3%A9alisations%20m%C3%A9caniques',
 'LAUAK%20FRANCE',
 'HEMERIA%20SERVICES',
 'COSMEPAR%20LABORATOIRE%20CONSEIL',
 'ACEBI',
 'CABINET%20BESSE%20TRANSPORTS%20RISQUES%20DIVERS',
 'CENTRE%20D%27ETUDES%20BIOLOGIQUES%20ET%20PHARMACEUTIQUES',
 'ATECA',
 'EUREP%20INDUSTRIES%20SA',
 'CORSE%20COMPOSITES%20AERONAUTIQUES',
 'HGH%20SYSTEMES%20INFRAROUGES',
 'TORAY%20CARBON%20FIBERS%20EUROPE',
 'EATON%20SAS',
 'TFCM',
 'EOLANE%20COMBREE',
 'LAMECO',
 'HALGAND',
 'SEREME',
 'PHATOPHY',
 'SAVIMEX',
 'GROUPE%20ROSSI%20AERO',
 'GMI%20AERO',
 'OMNIUM%20DES%20COMPOSANTS%20ELECTRONIQUES%20DE%20TECHNIQUES%20AVANCEES%20O.C.E.T.A.',
 'HITIM%20GROUP',
 'EUROPEENNE%20APPLICATION%20CRISTAUX%20LIQUIDES',
 'MOTEURS%

In [9]:
first_year = 2000

In [16]:
# assert (affiliation1_type in ['raw_affiliation_string', 'institutions.country_code', 'institutions.ror'])

def get_filename(affiliation1_str, affiliation1_type, affiliation2_str, affiliation2_type, thematic, random_size ):
    filename = f'cache/{normalize(affiliation1_str)}_{normalize(affiliation2_str)}_sample{random_size}_{normalize(thematic)}.json'.replace(' ', '_')
    return filename

In [17]:
BASE_URL = f"https://api.openalex.org/works?filter=publication_year:{first_year}-"

In [18]:
def get_search_field(f):
    if f in ['raw_affiliation_string']:
        return 'raw_affiliation_string.search'
    return f

def get_data(affiliation1_str, affiliation1_type, affiliation2_str, affiliation2_type, thematic ):
    url = BASE_URL + f",{get_search_field(affiliation1_type)}:{affiliation1_str}"
    if affiliation2_str:
        url += f",{get_search_field(affiliation2_type)}:{affiliation2_str}"
    if thematic:
        random_size = None
        url += f"&search={thematic}"
    else:
        random_size = 1000
        url += f"&sample={random_size}&seed=0"
        
    print(url)
    filename = get_filename(affiliation1_str, affiliation1_type, affiliation2_str, affiliation2_type, 
                            thematic, random_size )
    print(filename)
    try:
        data = json.load(open(filename, 'r'))
    except:
        results = []
        res = requests.get(url).json()
        nb_results = res['meta']['count']
        print(nb_results)
        nb_page = math.ceil(nb_results / res['meta']['per_page'])
        results = res['results']
        for p in range(2, nb_page + 1):
            print(p, end=',')
            res = requests.get(url+f"&page={p}").json()
            results += res['results']
        assert(len(results) == nb_results)
        data = {'results': results}
        json.dump(data, open(filename, 'w'))
    return data['results']

In [ ]:
all_works = pd.DataFrame()
thematic = None
for company in list_companies:
    data = get_data(company, 'raw_affiliation_string', 'france', 'raw_affiliation_string', thematic)
    df = pd.DataFrame(data)
    df['query'] = company
    all_works = pd.concat([all_works, df])

https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:GENINDEXE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/genindexe_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:AMBIENTIC,raw_affiliation_string.search:france&sample=1000&seed=0
cache/ambientic_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:HYDROCEAN,raw_affiliation_string.search:france&sample=1000&seed=0
cache/hydrocean_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:COVALIA%20INTERACTIVE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/covalia_20interactive_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:Aquitaine%20Science%20Transfert,raw_affiliation_string.search:france&sample=1000&seed=0
cache/aqu

https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:ALSEAMAR,raw_affiliation_string.search:france&sample=1000&seed=0
cache/alseamar_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:UNAC,raw_affiliation_string.search:france&sample=1000&seed=0
cache/unac_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:AKKA%20INGENIERIE%20PRODUIT,raw_affiliation_string.search:france&sample=1000&seed=0
cache/akka_20ingenierie_20produit_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:COMAT,raw_affiliation_string.search:france&sample=1000&seed=0
cache/comat_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:RENK%20FRANCE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/renk_20france_france_sample10

https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:ALLICE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/allice_france_sample1000_.json
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:AR2I%20SA%20ANALYSES%20RECHERCHES%20ET%20INNOVATION%20INSTRUMENTALE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/ar2i_20sa_20analyses_20recherches_20et_20innovation_20instrumentale_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:strate%20-%20Ecole%20de%20design,raw_affiliation_string.search:france&sample=1000&seed=0
cache/strate_20_-%_20ecole_20de_20design_france_sample1000_.json
1
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:QUALITE%20SECURITE%20ALIMENTATION,raw_affiliation_string.search:france&sample=1000&seed=0
cache/qualite_20securite_20alimentation_france_sample1000_.json
0


0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:NOVIMET,raw_affiliation_string.search:france&sample=1000&seed=0
cache/novimet_france_sample1000_.json
14
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:POTEZ%20COMPOSITES,raw_affiliation_string.search:france&sample=1000&seed=0
cache/potez_20composites_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:ACENSI%20SAS,raw_affiliation_string.search:france&sample=1000&seed=0
cache/acensi_20sas_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:TRINNOV%20AUDIO,raw_affiliation_string.search:france&sample=1000&seed=0
cache/trinnov_20audio_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:FLUOFARMA,raw_affiliation_string.search:france&sample=1000&seed=0
cache/fluofa

36
2,https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:KAPPA-SC,raw_affiliation_string.search:france&sample=1000&seed=0
cache/kappa_sc_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:CORHEM,raw_affiliation_string.search:france&sample=1000&seed=0
cache/corhem_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:LAETOLIS,raw_affiliation_string.search:france&sample=1000&seed=0
cache/laetolis_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:DELFMEMS,raw_affiliation_string.search:france&sample=1000&seed=0
cache/delfmems_france_sample1000_.json
10
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:SILVACO%20FRANCE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/silvaco_20france_france_sample1000_.json

...

32
2,https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:EFFINOV%20NUTRITION,raw_affiliation_string.search:france&sample=1000&seed=0
cache/effinov_20nutrition_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:ARCAL,raw_affiliation_string.search:france&sample=1000&seed=0
cache/arcal_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:NVH%20MEDICINAL,raw_affiliation_string.search:france&sample=1000&seed=0
cache/nvh_20medicinal_france_sample1000_.json
5
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:MDOLORIS%20MEDICAL%20SYSTEMS,raw_affiliation_string.search:france&sample=1000&seed=0
cache/mdoloris_20medical_20systems_france_sample1000_.json
4
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:KWAGA,raw_affiliation_string.search:france&sam

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:LABEXAN,raw_affiliation_string.search:france&sample=1000&seed=0
cache/labexan_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:SURVEY%20COPTER,raw_affiliation_string.search:france&sample=1000&seed=0
cache/survey_20copter_france_sample1000_.json
1
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:SYNERTIC,raw_affiliation_string.search:france&sample=1000&seed=0
cache/synertic_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:2%20S%20-%20COMPUTING,raw_affiliation_string.search:france&sample=1000&seed=0
cache/20s_20_-%_20computing_france_sample1000_.json
16
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:BIOMEOSTASIS,raw_affiliation_string.search:france&sample=1000&seed=0
cache/biome

3
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:FRETLINK,raw_affiliation_string.search:france&sample=1000&seed=0
cache/fretlink_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:LE%20SLIP%20FRANCAIS,raw_affiliation_string.search:france&sample=1000&seed=0
cache/le_20slip_20francais_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:ORNIKAR,raw_affiliation_string.search:france&sample=1000&seed=0
cache/ornikar_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:PERFECTSTAY.COM,raw_affiliation_string.search:france&sample=1000&seed=0
cache/perfectstay_com_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:PHENIX%20SAS,raw_affiliation_string.search:france&sample=1000&seed=0
cache/phenix_2

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:CAP%20LABO,raw_affiliation_string.search:france&sample=1000&seed=0
cache/cap_20labo_france_sample1000_.json
2
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:BARON%20CHRISTIAN,raw_affiliation_string.search:france&sample=1000&seed=0
cache/baron_20christian_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:EMSFACTORY,raw_affiliation_string.search:france&sample=1000&seed=0
cache/emsfactory_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:JALLAGEAS%2AJEREMY%20JEAN%20EDMOND,raw_affiliation_string.search:france&sample=1000&seed=0
cache/jallageas_2ajeremy_20jean_20edmond_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:SARL%20AIRESERVICES,raw_affiliation_strin

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:MIWEO,raw_affiliation_string.search:france&sample=1000&seed=0
cache/miweo_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:STIFFLER%2AROMAIN,raw_affiliation_string.search:france&sample=1000&seed=0
cache/stiffler_2aromain_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:MOOSTACK,raw_affiliation_string.search:france&sample=1000&seed=0
cache/moostack_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:SA%20AU%20CARRE,raw_affiliation_string.search:france&sample=1000&seed=0
cache/sa_20au_20carre_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:WITTY,raw_affiliation_string.search:france&sample=1000&seed=0
cache/witty_france_sample1000_.

In [66]:
all_works

,id,doi,title,display_name,relevance_score,publication_year,publication_date,ids,primary_location,host_venue,...,alternate_host_venues,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date,is_authors_truncated
0,https://openalex.org/W3184787821,https://doi.org/10.1080/20961790.2021.1886679,An inter-laboratory study of DNA-based identit...,An inter-laboratory study of DNA-based identit...,0.992930,2021,2021-07-19,{'openalex': 'https://openalex.org/W3184787821...,"{'is_oa': True, 'landing_page_url': 'https://d...","{'id': 'https://openalex.org/S4210169164', 'is...",...,"[{'id': 'https://openalex.org/S4210169164', 'd...","[https://openalex.org/W83837518, https://opena...","[https://openalex.org/W1525813537, https://ope...",https://api.openalex.org/works/W3184787821/ngrams,"{'The': [0, 34, 67, 181], 'probative': [1], 'v...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2023, 'cited_by_count': 1}]",2023-02-26T14:17:08.281707,2021-08-02,NaN
1,https://openalex.org/W3183842167,https://doi.org/10.1111/faf.12587,The early marine distribution of Atlantic salm...,The early marine distribution of Atlantic salm...,0.890448,2021,2021-07-05,{'openalex': 'https://openalex.org/W3183842167...,"{'is_oa': None, 'landing_page_url': 'https://d...","{'id': 'https://openalex.org/S103284624', 'iss...",...,"[{'id': 'https://openalex.org/S103284624', 'di...","[https://openalex.org/W186526879, https://open...","[https://openalex.org/W1796212355, https://ope...",https://api.openalex.org/works/W3183842167/ngrams,"{'The': [0, 94, 209], 'survival': [1], 'of': [...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2022, 'cited_by_count': 13}]",2023-02-16T22:46:26.093377,2021-08-02,NaN
2,https://openalex.org/W3017031654,https://doi.org/10.3897/zookeys.926.48679,"Painted black: Iguana melanoderma (Reptilia, S...","Painted black: Iguana melanoderma (Reptilia, S...",0.781384,2020,2020-04-13,{'openalex': 'https://openalex.org/W3017031654...,"{'is_oa': True, 'landing_page_url': 'https://d...","{'id': 'https://openalex.org/S199213172', 'iss...",...,"[{'id': 'https://openalex.org/S199213172', 'di...","[https://openalex.org/W618738618, https://open...","[https://openalex.org/W2003876219, https://ope...",https://api.openalex.org/works/W3017031654/ngrams,"{'The': [0, 226], 'Lesser': [1, 14, 29], 'Anti...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2023, 'cited_by_count': 1}, {'year':...",2023-02-20T13:08:38.448202,2020-04-24,NaN
3,https://openalex.org/W3164306728,https://doi.org/10.1007/s12686-021-01216-5,A genus-wide analysis of genetic variation to ...,A genus-wide analysis of genetic variation to ...,0.674782,2021,2021-05-22,{'openalex': 'https://openalex.org/W3164306728...,"{'is_oa': None, 'landing_page_url': 'https://d...","{'id': 'https://openalex.org/S147353869', 'iss...",...,"[{'id': 'https://openalex.org/S147353869', 'di...","[https://openalex.org/W618738618, https://open...","[https://openalex.org/W1994686255, https://ope...",https://api.openalex.org/works/W3164306728/ngrams,None,https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2022, 'cited_by_count': 5}]",2023-02-19T00:18:31.811719,2021-06-07,NaN
4,https://openalex.org/W2946260620,https://doi.org/10.11646/zootaxa.4608.2.1,A story of nasal horns: two new subspecies of ...,A story of nasal horns: two new subspecies of ...,0.099213,2019,2019-05-20,{'openalex': 'https://openalex.org/W2946260620...,"{'is_oa': None, 'landing_page_url': 'https://d...","{'id': 'https://openalex.org/S171471881', 'iss...",...,"[{'id': 'https://openalex.org/S171471881', 'di...",[https://openalex.org/W2473892951],"[https://openalex.org/W1972796382, https://ope...",https://api.openalex.org/works/W2946260620/ngrams,"{'The': [0, 126, 138, 312], 'Lesser': [1, 22, ...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2022, 'cited_by_count': 6}, {'year':...",2023-02-19T16:21:52.369343,2019-05-29,NaN
...,...,